In [ ]:
%reload_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
import run_yasa
import logging
import mne
import yasa
import os
import argparse
import os

import mne
import numpy as np
import pandas as pd
from brainflow.board_shim import BoardShim, BoardIds
from brainflow.data_filter import DataFilter
log = lambda msg: logging.info(msg)

In [ ]:
input_file = "C:\\dev\\play\\brainwave-data\\2024-12-11-21-31-37\\raw.post_human.csv"

In [ ]:
from sleep_events import convert_timestamps_to_uk

df = pd.read_csv(input_file)
convert_timestamps_to_uk(df, 'Timestamp', 'TimestampUK')

In [ ]:
df.head()

In [ ]:
from importlib import reload
import models.eeg_states.eeg_states
reload(models.eeg_states.eeg_states)
from models.eeg_states.eeg_states import load_and_prepare_settling_eeg_state_events
events = load_and_prepare_settling_eeg_state_events()

In [ ]:
events_for_day = events[events['dayAndNightOf'].astype(str) == "2024-12-11"]

In [20]:
# Ensure the 'SettlingEventMatchedIdx' column exists in the DataFrame
# if 'SettlingEventMatchedIdx' not in df.columns:
#     df['SettlingEventMatchedIdx'] = np.nan

# Fill NaN values in the 'SettlingEventMatchedIdx' column with a placeholder value
df['SettlingEventMatchedIdx'] = df['SettlingEventMatchedIdx'].fillna(-1)

In [21]:
# Reset the index of the events DataFrame and rename it to SettlingEventMatchedIdx
events = events.reset_index().rename(columns={'index': 'SettlingEventMatchedIdx'})

# Fill NaN values in the 'SettlingEventMatchedIdx' column with a placeholder value
# df['SettlingEventMatchedIdx'] = df['SettlingEventMatchedIdx'].fillna(-1).astype(int)

# Merge the DataFrames on the 'SettlingEventMatchedIdx' column
merged_df = df.merge(events, left_on='SettlingEventMatchedIdx', right_index=True, how='left')

# Display the merged DataFrame
merged_df.head()

,SettlingEventMatchedIdx,Stage,Confidence,Epoch,Timestamp,Source,Fpz-M1_Stage,Fpz-M1_Confidence,StageInt,Fpz-M1_eeg_abspow,...,BaseEvent,Preserve_SettlingEvent,Preserve_SettlingEventVersion,duration,TimestampUK_y,SinceUK,dayAndNightOf_y,Preserve_SettlingManualScore,Preserve_SettlingV4ManualScore,Preserve_TiredVsAlertManualScore
0,-1.0,W,0.994664,0,2024-12-11 21:31:37.562510967+00:00,Fpz-M1,W,0.994664,0,1.684779e-08,...,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN
1,-1.0,W,0.999240,1,2024-12-11 21:32:07.562510967+00:00,Fpz-M1,W,0.999240,0,1.629360e-09,...,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN
2,-1.0,W,0.998701,2,2024-12-11 21:32:37.562510967+00:00,Fpz-M1,W,0.998701,0,1.792676e-09,...,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN
3,-1.0,W,0.999397,3,2024-12-11 21:33:07.562510967+00:00,Fpz-M1,W,0.999397,0,1.365066e-09,...,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN
4,-1.0,W,0.999411,4,2024-12-11 21:33:37.562510967+00:00,Fpz-M1,W,0.999411,0,1.125421e-09,...,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN


In [22]:
merged_df[['Timestamp', 'SettlingManualScore', 'SettlingEventMatchedIdx', 'event', 'TimestampUK_y', 'SinceUK']]

,Timestamp,SettlingManualScore,SettlingEventMatchedIdx,event,TimestampUK_y,SinceUK
0,2024-12-11 21:31:37.562510967+00:00,NaN,-1.0,NaN,NaT,NaT
1,2024-12-11 21:32:07.562510967+00:00,NaN,-1.0,NaN,NaT,NaT
2,2024-12-11 21:32:37.562510967+00:00,NaN,-1.0,NaN,NaT,NaT
3,2024-12-11 21:33:07.562510967+00:00,NaN,-1.0,NaN,NaT,NaT
4,2024-12-11 21:33:37.562510967+00:00,NaN,-1.0,NaN,NaT,NaT
...,...,...,...,...,...,...
1145,2024-12-12 07:04:07.562510967+00:00,NaN,-1.0,NaN,NaT,NaT
1146,2024-12-12 07:04:37.562510967+00:00,NaN,-1.0,NaN,NaT,NaT
1147,2024-12-12 07:05:07.562510967+00:00,NaN,-1.0,NaN,NaT,NaT
1148,2024-12-12 07:05:37.562510967+00:00,NaN,-1.0,NaN,NaT,NaT


In [23]:
merged_df['SettlingManualScore'].value_counts()

SettlingManualScore
25.0     16
100.0     2
75.0      1
Name: count, dtype: int64

In [ ]:
events_for_day[['event', 'SinceUK', 'TimestampUK']]

In [ ]:
event_idx = 1255
events_for_idx = events_for_day.loc[[event_idx]]
# df_for_event = merged_df[merged_df['SettlingEventMatchedIdx'] == event_idx]
# df_for_event[['Timestamp', 'SettlingManualScore', 'SettlingEventMatchedIdx', 'event', 'TimestampUK_y', 'SinceUK']]

In [ ]:
events_for_idx

In [ ]:
from models.eeg_states.eeg_states import add_event_type

out = add_event_type(df, events_for_idx, True)

In [24]:
df_for_event = merged_df[merged_df['SettlingEventMatchedIdx'] == event_idx]
df_for_event[['Timestamp', 'SettlingManualScore', 'SettlingEventMatchedIdx', 'event', 'TimestampUK_y', 'SinceUK']]


,Timestamp,SettlingManualScore,SettlingEventMatchedIdx,event,TimestampUK_y,SinceUK
97,2024-12-11 22:20:07.562510967+00:00,100.0,1255.0,NaN,NaT,NaT


KeyError: 'SettlingEventMatchedIdx'

In [33]:
matched_events_for_day = events_for_day.merge(df, left_index=True, right_on='SettlingEventMatchedIdx', how='left', indicator=True)
matched_events_for_day

,event,phase,version,note,source,BaseEvent,Preserve_SettlingEvent,Preserve_SettlingEventVersion,duration,TimestampUK_x,...,minsUntilWake,SettlingEventMatchedIdx,SettlingEvent,SettlingEventVersion,SettlingManualScore,SettlingV4ManualScore,TiredVsAlertManualScore,SettlingTiredVsWiredPrediction,SettlingScorePrediction,_merge
NaN,tired but alert,settling,v4,,NaN,tired but alert,tired but alert,v4,NaT,2024-12-11 20:29:50.297000+00:00,...,NaN,1243.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
NaN,tired but alert,settling,v4,,NaN,tired but alert,tired but alert,v4,NaT,2024-12-11 20:37:30.274000+00:00,...,NaN,1244.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
NaN,wired or alert,settling,v4,,NaN,wired or alert,wired or alert,v4,NaT,2024-12-11 21:05:02.078000+00:00,...,NaN,1245.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
NaN,wired or alert,settling,v4,NaN,homeAssistant,wired or alert,wired or alert,v4,NaT,2024-12-11 21:25:35.529000+00:00,...,NaN,1246.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
6.0,tired but wired,settling,v4,NaN,homeAssistant,tired but wired,tired but wired,v4,NaT,2024-12-11 21:34:53.350000+00:00,...,562.373958,1247.0,tired but wired,v4,25.0,25.0,0.0,1.000000e+00,1.0,both
7.0,tired but wired,settling,v4,NaN,homeAssistant,tired but wired,tired but wired,v4,NaT,2024-12-11 21:35:35.915000+00:00,...,561.873958,1248.0,tired but wired,v4,25.0,25.0,0.0,1.000000e+00,1.0,both
8.0,tired but wired_long,settling,v4,NaN,homeAssistant,tired but wired,tired but wired,v4,0 days 00:01:35.117000,2024-12-11 21:37:11.032000+00:00,...,561.373958,1249.0,tired but wired,v4,25.0,25.0,0.0,1.000000e+00,1.0,both
9.0,tired but wired_long,settling,v4,NaN,homeAssistant,tired but wired,tired but wired,v4,0 days 00:01:35.117000,2024-12-11 21:37:11.032000+00:00,...,560.873958,1249.0,tired but wired,v4,25.0,25.0,0.0,1.000000e+00,1.0,both
10.0,tired but wired_long,settling,v4,NaN,homeAssistant,tired but wired,tired but wired,v4,0 days 00:01:35.117000,2024-12-11 21:37:11.032000+00:00,...,560.373958,1249.0,tired but wired,v4,25.0,25.0,0.0,1.000000e+00,1.0,both
11.0,tired but wired_long,settling,v4,NaN,homeAssistant,tired but wired,tired but wired,v4,0 days 00:01:35.117000,2024-12-11 21:37:11.032000+00:00,...,559.873958,1249.0,tired but wired,v4,25.0,25.0,0.0,1.000000e+00,1.0,both


In [34]:
matched_events_for_day['_merge'].value_counts()

_merge
both          19
left_only      5
right_only     0
Name: count, dtype: int64

In [36]:
events_that_do_not_match_epochs = matched_events_for_day[matched_events_for_day['_merge'] == 'left_only']

KeyError: 'TimestampUK'

In [39]:
events_that_do_not_match_epochs[['TimestampUK_x']]

,TimestampUK_x
NaN,2024-12-11 20:29:50.297000+00:00
NaN,2024-12-11 20:37:30.274000+00:00
NaN,2024-12-11 21:05:02.078000+00:00
NaN,2024-12-11 21:25:35.529000+00:00
NaN,2024-12-11 22:20:14.407000+00:00
